# 組合せ最適化問題
組合せ最適化問題は、多くの選択肢からベストな答えを選ぶ問題で、社会問題をうまく定式化し、最小値問題を解くことで最適な組合せを得ることができます。

## 今回学ぶこと
1. 問題の定式化とハミルトニアン
2. イジングモデルとQUBO

Blueqatのインストール
pipからBlueqatをインストールします。

In [1]:
!pip install blueqat

----

## イジングモデル
量子コンピュータで組合せ最適化問題を解く際には通常イジングモデルと呼ばれる物理統計モデルを利用します。イジングモデルでは、量子ビットが並んでおり、接続された量子ビット同士の間の相互作用$J_{ij}$と量子ビット単体にかかる局所磁場$h_i$の値を設定しながら問題の定式化を行います。

$$
H = \sum h_i \sigma^z_i + \sum J_{ij}\sigma^z_i \sigma^z_j
$$

## ハミルトニアンと固有値
イジングモデルを利用した組み合わせ最適化問題では、定式化をハミルトニアンと呼ばれる行列に落とし込みます。そして、その固有値を求めることで最小値問題を解きます。ハミルトニアンの定式化にはZ演算子を利用します。定式化の例としては例えば、

```python
h = -Z(0) - Z(0)*Z(1)
```

のようにします。Zの後ろの数字は、量子ビットの通し番号を表します。0番目と1番目の量子ビットの２つを使っています。問題設定で大事なのは、Zの前の係数です。

Z(0)の前は-1のバイアス、Z(0)*Z(1)の前は-1のウェイトが設定されています。Zは期待値として-1か+1のどちらかをとります。hはより小さい値を取ると正解になります。最終的な答えは、Z(0),Z(1)の値で場合わけすると、

Z(0) | Z(1) | h
--:|:----:|:--
-1|-1|0
-1|1|2
1|-1|0
1|1|-2

上記の表で最小となるZ(0)=1,Z(1)=1の時-2となるものを計算で求めます。

## QUBO
上記イジングでは量子ビットのとりうる値を+1と-1としました。産業界ではそれでは扱いづらいので、QUBO(キューボ)と呼ばれる01の値をとる定式化を利用するのが一般的です。幸いイジングとQUBOは自動的に変換ができますので、01を使っても組合せ最適化問題としては問題がありません。

係数は様々な呼び方がありますが「バイアス」と「ウェイト（結合荷重）」と呼びます。定式化で作るのは「コスト関数」と呼びたいと思います。

-1と+1でかかれたイジング式を0と1で書かれたQUBO式に変換するには、イジングのZを下記のように変換するだけでできます。

$$
q = \frac{Z + 1}{2}
$$

これで、-1の時が0に、1の時は1のままで変換されます。定式化は社会問題をQUBO形式で、コスト関数を作ることで実現でき、コスト関数は01の値をとる量子ビットに設定するバイアスとウェイトを設定することで実現できます。

Blueqatでは、下記のように自動的にQUBO式からイジングに変換してくれる機能が搭載されています。

In [1]:
from blueqat.pauli import qubo_bit as q

h = -3*q(0)-3*q(1)-2*q(0)*q(1)

## QUBO matrix
量子アニーリングを解く際にはQUBO matrixと呼ばれるバイアスを対角成分に、ウェイトを非対角成分においた上三角行列を使うことがあります。量子アニーリングマシンの実機は2体相互作用までが実装されるので、QUBO matrixがよく使われます。対称行列ですが、通常は上半分の上三角行列を使います。

$$QUBO = 
\begin{bmatrix}
h_0 & J_{01} & J_{02} & ... & J_{0n}\\
0   & h_1    & J_{12} & ... & J_{1n}\\
\vdots&&&&\vdots\\
0&0&0&...&h_n
\end{bmatrix}
$$

Blueqatでは、QUBO matrixからZ演算子に変換してくれる機能が搭載されています。

In [2]:
import blueqat.opt as wq

qubo_matrix = [[-1, 2, 2, 2],[0, -1, 2, 2],[0, 0, -1, 2],[0, 0, 0, -1]]
qubo = wq.pauli(qubo_matrix)

print(qubo)

0.5*Z[0]*Z[1] + 1.0*I - Z[2] - Z[0] + 0.5*Z[0]*Z[2] - Z[3] + 0.5*Z[0]*Z[3] - Z[1] + 0.5*Z[1]*Z[2] + 0.5*Z[1]*Z[3] + 0.5*Z[2]*Z[3]


## QUBOとグラフ問題
QUBOはグラフ問題としてもよく表現されます。量子ビットをノード、接続をエッジとしてみた時、QUBO matrixはグラフネットワークの接続と重み付けを表現しているともみて取れます。

----

## 例題：イジング+VQE
イジング問題を固有値問題として解く際には、VQEとQAOAが利用できます。VQEだけでは効率的には組合せ最適化問題は解けませんが、簡単な量しか色を使って問題を解いてみます。

問題の定式化のハミルトニアンには上記のものを利用します。VQEで解くにはAnsatzと呼ばれる量子回路を準備する必要がありますが、今回はa,b,c,dの4つの角度を持った簡単な2量子ビットの回路を準備します。

In [3]:
import numpy as np
from blueqat import Circuit
from blueqat.pauli import X, Y, Z, I
from blueqat.vqe import AnsatzBase, Vqe

class OneQubitAnsatz(AnsatzBase):
    def __init__(self, hamiltonian):
        super().__init__(hamiltonian.to_expr(), 4)
        self.step = 1

    def get_circuit(self, params):
        a, b, c, d = params
        return Circuit().ry(a)[0].rz(b)[0].ry(c)[1].rz(d)[1]

h = -Z(0) - Z(0)*Z(1)
runner = Vqe(OneQubitAnsatz(h))
result = runner.run()

print('Result by VQE')
print(runner.ansatz.get_energy_sparse(result.circuit))

Result by VQE
-1.999998714607245


約-2が答えとして出てきましたので、正解です。今回は例題として任意の量子状態を使いましたが、通常はQAOAを使います。そして実際にはより大きな問題を解きます。

上記問題は役にたたなさそうですが、Z(0)をAさん、Z(1)をBさんに見立てて、Aさんはグループ１に属し、BさんはAさんと同じグループに属するという条件をつけた分類問題と同じです。

ただ、このままでは毎回問題を解くのが大変なので様々な工夫が必要です、それをみていきましょう。

## 例題：イジング+QAOA

上記の問題をQAOAで解いてみます。QAOAは効率的に問題を解くAnsatzがありますのでそれを使います。ハミルトニアンとステップ数を入れれば完了です。

In [10]:
from blueqat import vqe

h = -Z(0) - Z(0)*Z(1)

step = 2
runner = Vqe(vqe.QaoaAnsatz(h, step))
result = runner.run()

print(runner.ansatz.get_energy_sparse(result.circuit))

-1.7919100877330898


## 例題：QUBO + 各種計算
今回はQUBOで書かれた式を準備して、これを解いてみたいと思います。

```
h = -3*q(0)-3*q(1)-2*q(0)*q(1)
```

これは、q(0)=1,q(1)=1の時に最小値-3-3-2=-8を取ります。上記のコスト関数は以前のように-1と+1ではなく、0と1で考えることができるので簡単です。QAOAで解いてみます。

In [6]:
from blueqat import vqe
from blueqat.pauli import qubo_bit as q

h = -3*q(0)-3*q(1)-2*q(0)*q(1)
step = 2

result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((1, 1), 0.9571688060369407), ((0, 0), 0.02264678008376087), ((0, 1), 0.010092206939649306), ((1, 0), 0.010092206939649295))


答えが(1,1)となりました。次に量子アニーリングに近い形でQUBO matrixを使ってSAシミュレータで解いてみます。上記のハミルトニアンのバイアスとウェイトをQUBO matrixの形に落とし込みます。

In [7]:
qubo = [[-3, -2],
        [0 , -3]]

これをそのまま計算にかけます。

In [8]:
import blueqat.opt as wq

a = wq.opt()
a.qubo = qubo
a.sa()

[1, 1]

これも同じ答えが出ました。QUBO matrixをZ演算子に直してQAOAで計算してみます。

In [9]:
step = 2
h = wq.pauli(qubo)

result = vqe.Vqe(vqe.QaoaAnsatz(h, step)).run()
print(result.most_common(12))

(((1, 1), 0.9872629031715499), ((0, 0), 0.01265996782202954), ((0, 1), 3.856450321039442e-05), ((1, 0), 3.856450321039259e-05))


このようにQUBO matrixやZ演算子を使って計算ができました。